In [ ]:
"""
During the initial phase of the decoder model building,
I tried the following parameters:
- using the whole spectrogram (11 uniformly spaced frequency bins) for model input
- using normalized LMP
- using z-scored bands

They are not used any more because they do not improve the model accuracy while either
being computationally heavier or physically less relevant (hence more difficult to justify
their usage).
Codes for generating these data have been commented out
"""

In [ ]:
%load_ext autoreload
%autoreload 2
import os, sys

import numpy as np

from matplotlib import pyplot as plt
from matplotlib_settings import set_plot_settings, reset_plot_settings

# Set the plot settings
set_plot_settings()

# import global variables
from utils_motor_global import *

from utils_motor_sigproc import normalize_band
# from utils_motor_sigproc import normalize_spect

from utils_motor_model import make_full_t_model_input

ROOT_SAVE_DIR = f'{MODEL_INPUT_DIR}/scalogram_matrix'

# model input time resolution. 10 ms = T_DF_MATRIX*T_STEP_matrix

In [ ]:
""" save M1, S1 channels.. not required for model """
from utils_motor_plot import draw_CS_boundary, CS_COLS
all_chs = np.zeros((256, ))
M1_chs, S1_chs = [], []

for row, cols in enumerate(CS_COLS):
    for col in range(cols[0]):
        M1_chs.append(row*16 + col)
    for col in range(cols[1], 16):
        S1_chs.append(row*16 + col)

M1_chs = np.array(M1_chs)
S1_chs = np.array(S1_chs)

all_chs[M1_chs] = -1
all_chs[S1_chs] = 1

# fig, ax = plt.subplots()
# ax.imshow(all_chs.reshape(16, -1))
# ax.set_xticks([])
# ax.set_yticks([])

# draw_CS_boundary(ax)

np.save(f'{ROOT_SAVE_DIR}/M1_channels.npy', M1_chs)
np.save(f'{ROOT_SAVE_DIR}/S1_channels.npy', S1_chs)

In [ ]:
""" load common channels """
common_chs = np.load(f'{ROOT_SAVE_DIR}/common_good_channels.npy') # copy-pasted manually.

""" channel band mean and standard deviation """
spect_mu  = np.load(f'{BAND_MU_SIGMA_DIR}/spect_mu.npy')
lmp_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_mu.npy')
lfs_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_mu.npy')
beta_mu   = np.load(f'{BAND_MU_SIGMA_DIR}/beta_mu.npy')
lga_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/lga_mu.npy')
hga_mu    = np.load(f'{BAND_MU_SIGMA_DIR}/hga_mu.npy')

spect_sigma  = np.load(f'{BAND_MU_SIGMA_DIR}/spect_sigma.npy')
lmp_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_sigma.npy')
lfs_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/lmp_sigma.npy')
beta_sigma   = np.load(f'{BAND_MU_SIGMA_DIR}/beta_sigma.npy')
lga_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/lga_sigma.npy')
hga_sigma    = np.load(f'{BAND_MU_SIGMA_DIR}/hga_sigma.npy')

In [ ]:
for key in SESSION_KEYS:
    load_dir = f'{REC_DIR}/6_truncate/{key}'
    if not os.path.exists(load_dir):
        continue
    print(f'processing {key}...')

    save_data_dir = f'{ROOT_SAVE_DIR}/{key}'
    if not os.path.exists(save_data_dir):
        os.makedirs(save_data_dir)

    """ load data """
    # recording data
    good_chs    = np.load(f'{load_dir}/good_channels_{key}.npy')
    spect       = np.load(f'{load_dir}/spect_{key}.npy'     ) 
    lmp_data    = np.load(f'{load_dir}/lmp_{key}.npy'       ) 
    spect_t     = np.load(f'{load_dir}/spect_t_{key}.npy'   ) 
    
    spect_f     = np.load(f'{load_dir}/spect_f_{key}.npy'   ) 
    beta_fidxs  = np.load(f'{load_dir}/beta_fidxs_{key}.npy') 
    lga_fidxs   = np.load(f'{load_dir}/lga_fidxs_{key}.npy' ) 
    hga_fidxs   = np.load(f'{load_dir}/hga_fidxs_{key}.npy' ) 
    
    spect_lfs  = np.sum(spect[:,:,[0]], axis=2)
    spect_beta = np.sum(spect[:,:,beta_fidxs], axis=2)
    spect_lga  = np.sum(spect[:,:, lga_fidxs], axis=2)
    spect_hga  = np.sum(spect[:,:, hga_fidxs], axis=2)
    
    # motion data has been already interpolated to model input time points
    model_t     = np.load(f'{MODEL_INPUT_DIR}/motion/{key}/model_t_{key}.npy')

    """ normalize """
    # norm_lmp   = normalize_band(good_chs, lmp_data, lmp_mu, lmp_sigma, sel_zscore=False)
    norm_lfs   = normalize_band(good_chs, spect_lfs, lfs_mu, lfs_sigma, sel_zscore=False)
    norm_beta  = normalize_band(good_chs, spect_beta, beta_mu, beta_sigma, sel_zscore=False)
    norm_lga   = normalize_band(good_chs, spect_lga, lga_mu, lga_sigma, sel_zscore=False)
    norm_hga   = normalize_band(good_chs, spect_hga, hga_mu, hga_sigma, sel_zscore=False)

    # spectrogram is not used any more for building model input
    # norm_spect = normalize_spect(good_chs, spect, spect_mu, spect_sigma, sel_zscore=False)

    """ z-score """
    zs_lmp  = normalize_band(good_chs, lmp_data, lmp_mu, lmp_sigma, sel_zscore=True)
    # zs_lfs  = normalize_band(good_chs, spect_lfs, lfs_mu, lfs_sigma, sel_zscore=True)
    # zs_beta = normalize_band(good_chs, spect_beta, beta_mu, beta_sigma, sel_zscore=True)
    # zs_lga  = normalize_band(good_chs, spect_lga, lga_mu, lga_sigma, sel_zscore=True)
    # zs_hga  = normalize_band(good_chs, spect_hga, hga_mu, hga_sigma, sel_zscore=True)

    # spectrogram is not used any more for building model input
    # zs_spect = normalize_spect(good_chs, spect, spect_mu, spect_sigma, sel_zscore=True)

    """ remove non-common channels """
    common_ch_idxs = [np.where(good_chs == ch)[0][0] for ch in common_chs]
    assert np.array_equal(common_chs, good_chs[common_ch_idxs])

    # norm_lmp    =  norm_lmp  [common_ch_idxs]
    norm_lfs    =  norm_lfs  [common_ch_idxs]
    norm_beta   =  norm_beta [common_ch_idxs]
    norm_lga    =  norm_lga  [common_ch_idxs]
    norm_hga    =  norm_hga  [common_ch_idxs]
    # norm_spect  =  norm_spect[common_ch_idxs]

    zs_lmp      =  zs_lmp    [common_ch_idxs]
    # zs_lfs      =  zs_lfs    [common_ch_idxs]
    # zs_beta     =  zs_beta   [common_ch_idxs]
    # zs_lga      =  zs_lga    [common_ch_idxs]
    # zs_hga      =  zs_hga    [common_ch_idxs]
    # zs_spect    =  zs_spect  [common_ch_idxs]

    """ check that model input matrix length matches motion data length """
    idx0 = np.where(spect_t >= model_t[0])[0][0]
    idx1 = np.where(spect_t <= model_t[-1])[0][-1] + 1
    assert len(spect_t[idx0:idx1]) == len(model_t)

    """ model input params """
    dt = spect_t[1] - spect_t[0]
    tau_idx0 = round(-FULL_TAU_START/dt)
    tau_idx1 = round(FULL_TAU_END/dt)
    # model_f_idxs = np.arange(0, len(spect_f))

    """ build model input matrices """
    # norm_spect_matrix = make_full_t_model_input(norm_spect.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX, model_f_idxs)
    norm_lfs_matrix   = make_full_t_model_input(norm_lfs.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    norm_beta_matrix  = make_full_t_model_input(norm_beta.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    norm_lga_matrix   = make_full_t_model_input(norm_lga.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    norm_hga_matrix   = make_full_t_model_input(norm_hga.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)

    zs_lmp_matrix     = make_full_t_model_input(zs_lmp.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    # zs_lfs_matrix     = make_full_t_model_input(zs_lfs.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    # zs_beta_matrix    = make_full_t_model_input(zs_beta.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    # zs_lga_matrix     = make_full_t_model_input(zs_lga.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)
    # zs_hga_matrix     = make_full_t_model_input(zs_hga.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX)

    """ dimension check """
    # assert norm_spect_matrix.shape[0] == len(model_t[::T_DF_MATRIX])
    assert zs_lmp_matrix.shape[0] == len(model_t[::T_DF_MATRIX])

    """ save """
    # carry over:
    np.save(f'{save_data_dir}/model_t_{key}.npy', model_t) 
    np.save(f'{save_data_dir}/spect_t_{key}.npy', spect_t) 
    np.save(f'{save_data_dir}/spect_f_{key}.npy', spect_f) 

    # np.save(f'{save_data_dir}/norm_spect_matrix_{key}.npy', norm_spect_matrix)
    np.save(f'{save_data_dir}/norm_lfs_matrix_{key}.npy', norm_lfs_matrix)
    np.save(f'{save_data_dir}/norm_beta_matrix_{key}.npy', norm_beta_matrix)
    np.save(f'{save_data_dir}/norm_lga_matrix_{key}.npy', norm_lga_matrix)
    np.save(f'{save_data_dir}/norm_hga_matrix_{key}.npy', norm_hga_matrix)
    
    np.save(f'{save_data_dir}/zs_lmp_matrix_{key}.npy', zs_lmp_matrix)
    
    # np.save(f'{save_data_dir}/zs_lfs_matrix_{key}.npy', zs_lfs_matrix)
    # np.save(f'{save_data_dir}/zs_beta_matrix_{key}.npy', zs_beta_matrix)
    # np.save(f'{save_data_dir}/zs_lga_matrix_{key}.npy', zs_lga_matrix)
    # np.save(f'{save_data_dir}/zs_hga_matrix_{key}.npy', zs_hga_matrix)

    # build this too if storage space allows
    # zs_spect_matrix = make_full_t_model_input(zs_spect.astype(dtype='float16'), tau_idx0, tau_idx1, T_DF_MATRIX, model_f_idxs)
    # np.save(f'{save_data_dir}/zs_spect_matrix_{key}.npy', zs_spect_matrix)

In [ ]:
# for key in SESSION_KEYS:
#     fn0 = f'norm_lfs_scalo_{key}.npy'
#     fn1 = f'norm_lfs_matrix_{key}.npy'
# 
#     # fn0 = f'norm_beta_scalo_{key}.npy'
#     # fn1 = f'norm_beta_matrix_{key}.npy'
# 
#     # fn0 = f'norm_lga_scalo_{key}.npy'
#     # fn1 = f'norm_lga_matrix_{key}.npy'
# 
#     # fn0 = f'norm_hga_scalo_{key}.npy'
#     # fn1 = f'norm_hga_matrix_{key}.npy'
# 
#     # fn0 = f'zs_lmp_scalo_{key}.npy'
#     # fn1 = f'zs_lmp_matrix_{key}.npy'
# 
#     # fn0 = f'scalo_t_{key}.npy'
#     # fn1 = f'model_t_{key}.npy'
# 
#     # fn0 = f'spect_t_{key}.npy'
#     # fn1 = f'spect_t_{key}.npy'
#     
#     # fn0 = f'spect_f_{key}.npy'
#     # fn1 = f'spect_f_{key}.npy'
# 
#     dir0 = f'./combined_preprocessed_v3/4_scalogram/{key}'
#     dir1 = f'./model_input/scalogram_matrix/{key}'
# 
#     X0 = np.load(f'{dir0}/{fn0}')
#     X1 = np.load(f'{dir1}/{fn1}')
#     assert np.allclose(X0, X1, equal_nan=True)